In [8]:
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

In [1]:
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
def append_ext(fn):
    return fn+".png"
base_path='data/024/'
traindf=pd.read_csv(base_path+"trainLabels.csv",dtype=str)
testdf=pd.read_csv(base_path+"sampleSubmission.csv",dtype=str)
traindf["id"]=traindf["id"].apply(append_ext)
testdf["id"]=testdf["id"].apply(append_ext)
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [3]:
train_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=base_path+"train/",
    x_col="id",
    y_col="label",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(32, 32))
valid_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=base_path+"train/",
    x_col="id",
    y_col="label",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(32, 32))
test_datagen = ImageDataGenerator(rescale=1. / 255.)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory=base_path+"test/",
    x_col="id",
    y_col=None,
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(32, 32))

Found 37500 validated image filenames belonging to 10 classes.
Found 12500 validated image filenames belonging to 10 classes.
Found 300000 validated image filenames.


In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1171/1171 [==============================] - 43s 36ms/step - loss: 1.9170 - accuracy: 0.2978 - val_loss: 1.8686 - val_accuracy: 0.3962
Epoch 2/10
1171/1171 [==============================] - 12s 10ms/step - loss: 1.6008 - accuracy: 0.4171 - val_loss: 1.5148 - val_accuracy: 0.4783
Epoch 3/10
1171/1171 [==============================] - 12s 10ms/step - loss: 1.4576 - accuracy: 0.4750 - val_loss: 0.9220 - val_accuracy: 0.5126
Epoch 4/10
1171/1171 [==============================] - 12s 10ms/step - loss: 1.3630 - accuracy: 0.5132 - val_loss: 1.1636 - val_accuracy: 0.5554
Epoch 5/10
1171/1171 [==============================] - 12s 10ms/step - loss: 1.2870 - accuracy: 0.5441 - val_loss: 1.1081 - val_accuracy: 0.5852
Epoch 6/10
1171/1171 [==============================] - 12s 10ms/step - loss: 1.2200 - accuracy: 0.5690 - val_loss: 1.3614 - val_accuracy: 0.5932
Epoch 7/10
1171/1171 [==============================] - 12s 10ms/step - loss

In [6]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_TEST)

[0.8667998909950256, 0.6626930236816406]